In [4]:
import pandas as pd
from simplet5 import SimpleT5
from sklearn.model_selection import train_test_split
import torch

# Step 1: Read the CSV file
data = pd.read_csv('testing1.csv')

# Ensure the DataFrame has the required columns
assert 'source_text' in data.columns and 'target_text' in data.columns, "CSV must contain 'source_text' and 'target_text' columns."

# Step 2: Split the data into training and testing sets (80% training, 20% testing)
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# No need to rename columns as they already have the correct names

# Step 3: Initialize SimpleT5
model = SimpleT5()
model.from_pretrained("t5", "t5-small")

# Step 4: Train the model (without GPU)
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=16,
            max_epochs=5,
            use_gpu=True)  # Set use_gpu=False for CPU usage

# Step 5: Set the device manually
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.model.to(device)

# Step 6: Make predictions (assuming you have input text)
def generate_text(input_text):
    input_ids = model.tokenizer.encode(input_text, return_tensors='pt', max_length=128, truncation=True).to(device)
    outputs = model.model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
    return model.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example prediction
output = generate_text("When assembling furniture or household items, what power tool is recommended?")
print(output)


INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


DRILL DRIVER


In [ ]:
pip install simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_

In [7]:
import pandas as pd
from simplet5 import SimpleT5
from sklearn.model_selection import train_test_split
import torch
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import numpy as np

# Download the NLTK data for BLEU
nltk.download('punkt')

# Step 1: Read the CSV file
data = pd.read_csv('testing1.csv')

# Ensure the DataFrame has the required columns
assert 'source_text' in data.columns and 'target_text' in data.columns, "CSV must contain 'source_text' and 'target_text' columns."

# Step 2: Split the data into training and testing sets (80% training, 20% testing)
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Step 3: Initialize SimpleT5
model = SimpleT5()
model.from_pretrained("t5", "t5-small")

# Step 4: Train the model (without GPU)
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=256,  # Increased max token length
            target_max_token_len=128,  # Increased max token length
            batch_size=8,              # Reduced batch size if memory is an issue
            max_epochs=10,             # Increased number of epochs
            use_gpu=True)             # Set use_gpu=True if you have a GPU

# Step 5: Set the device manually
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.model.to(device)

# Step 6: Make predictions and calculate BLEU score
def generate_text(input_text):
    input_ids = model.tokenizer.encode(input_text, return_tensors='pt', max_length=256, truncation=True).to(device)
    outputs = model.model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    return model.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to calculate BLEU score for the test set
def calculate_bleu_score(test_df):
    references = []
    candidates = []
    bleu_scores = []
    for _, row in test_df.iterrows():
        source_text = row['source_text']
        target_text = row['target_text']
        predicted_text = generate_text(source_text)

        # Tokenize the target and predicted text
        reference = [nltk.word_tokenize(target_text)]
        candidate = nltk.word_tokenize(predicted_text)

        # Calculate BLEU score
        bleu_score = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu_score)

        references.append(reference)
        candidates.append(candidate)

        print(f"Source: {source_text}")
        print(f"Target: {target_text}")
        print(f"Predicted: {predicted_text}")
        print(f"BLEU score: {bleu_score}\n")

    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    corpus_bleu_score = corpus_bleu(references, candidates)
    return average_bleu_score, corpus_bleu_score, bleu_scores

# Calculate the BLEU score for the test set
average_bleu_score, corpus_bleu_score, bleu_scores = calculate_bleu_score(test_df)
print(f"Average sentence-level BLEU score: {average_bleu_score}")
print(f"Corpus-level BLEU score: {corpus_bleu_score}")
print(f"BLEU score standard deviation: {np.std(bleu_scores)}")
print(f"BLEU score median: {np.median(bleu_scores)}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Source: What power tool is preferred by contractors for concrete drilling and chiseling?
Target: ROTATORY HAMMER DRILLER
Predicted: DEMOLITION HAMMER
BLEU score: 9.291879812217675e-232

Source: For hobbyists interested in construction projects involving demolition, what power tool is versatile?
Target: DEMOLITION HAMMER
Predicted: DEMOLITION HAMMER
BLEU score: 1.491668146240062e-154

Source: Which power tool is ideal for tasks like installing shelves and racks on walls?
Target: IMPACT DRILLER
Predicted: DRILLS & DRIVERS
BLEU score: 0

Source: Can you identify a powerful device used by construction crews to penetrate walls and concrete slabs?
Target: DEMOLITION HAMMER
Predicted: DEMOLITION HAMMER
BLEU score: 1.491668146240062e-154

Source: Describe the equipment crucial for controlled demolition activities in urban environments.
Target: DEMOLITION HAMMER
Predicted: DEMOLITION HAMMER
BLEU score: 1.491668146240062e-154

Source: Which power tool is beginner-friendly for basic DIY tasks?
Ta